In [58]:
import numpy as np
import datetime as dt
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, distinct, and_

from flask import Flask, jsonify

In [2]:
#################################################
# Database Setup
#################################################
#rds_connection_string = "ETLprj:ScottSaid110!@localhost:5432/migration_db"
rds_connection_string = "immigration_cnn:@localhost:5432/migration_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

#'immigration_cnn

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the table
details = Base.classes.details
regional = Base.classes.regional
regions = Base.classes.regions
counties = Base.classes.counties
states = Base.classes.states


In [100]:
Base.classes.keys()

['details', 'regional', 'regions', 'counties', 'states']

In [101]:
pd.read_sql_query('select * from Regional', con=engine).head()

id           birth_country  age_0_5  age_6_10  age_11_15  age_16_20  \
0  17                      US        0         0          0          0   
1  18                  Mexico   928523    775875    1485420    2238925   
2  19  Eastand Southeast Asia  1500243    946080     926337     941610   
3  20            Central Asia    38881     21083      20612      17039   
4  21              South Asia  1148791    589390     446589     470990   

   age_20_plus  grade_9th  grade_12th  hs_dropout  ...  cleaning_and_maint  \
0            0    4380278    11086857      587488  ...             4183259   
1      5753368    3557555     1818941       26761  ...              787212   
2      4334258     761904      471609        3533  ...              146691   
3        34239       2938        2164           0  ...                2776   
4      1013222     162268      146226        2590  ...               27050   

   other_services     sales  administrative  farming_and_fishing  \
0         8377235  17391363        19613568               916886   
1          216708    494685          482108               473083   
2          388236    525913          559092                15351   
3            5757      7961            7078                    0   
4           62512    273505          157368                 3970   

   construction  manufacturing  transportation  military  unemployed  
0       7269547       11784060        12504870    556905     1317281  
1       1316937        1001810          851937      6356       68478  
2         87448         465487          297208     10277       39188  
3          4097           5342           11118        83         509  
4         18523          92958          149425      1178       24066  

[5 rows x 39 columns]

In [102]:
demography_json = [

    {demography:"Age",
    labels: ['Age_0-5', "Age_6-10", "Age_11-15", "Age_16-20", "Age_20+"]
    },
   
    {
        US:	[0.000, 0.000, 0.000, 0.000, 0.000],
        Mexico:[0.083, 0.0694, 0.1328, 0.2002, 0.5145],
        Eastand_Southeast_Asia: [0.1735, 0.1094, 0.1071, 0.1089, 0.5012],
        Central_Asia: [0.2949, 0.1599, 0.1563, 0.1292, 0.2597],
        South_Asia: [0.3131, 0.1606, 0.1217, 0.1284, 0.2762],
        Oceania: [0.248, 0.1147, 0.1338, 0.1386, 0.3648 ],
        Europe: [0.1356, 0.074, 0.0855, 0.1308, 0.5741],
        Canada_and_Other_North_America: [0.1833, 0.0816, 0.0761, 0.1053, 0.5537],
        Caribbean: [0.1722, 0.1292, 0.1053, 0.1073, 0.486],
        Central_America: [0.1832, 0.1023, 0.1558, 0.1496, 0.4092],
        South_America: [0.2139, 0.0886, 0.111, 0.1896, 0.3969],
        Middle_EastNorth_Africa: [0.2426, 0.1547, 0.103, 0.1149, 0.3847],
        SubSaharan_Africa: [0.2695, 0.1866, 0.1626, 0.1492, 0.2321],


    }
]

SyntaxError: invalid character in identifier (<ipython-input-102-4a440c685a88>, line 9)

In [103]:
demography = 'age'

def migration_data(demography):

    session = Session(engine)

    stmt = session.query(regional).statement
    df = pd.read_sql_query(stmt, session.bind)

    session.close

    meta_dict = {'demography':demography}

    if demography == 'age':
    
        demography_df = df.iloc[:,1:7].T
        meta_dict['labels'] =  labels[0:5]
    
    elif demography == 'education':
        demography_df = df.iloc[:,np.r_[1, 7:16]].T
        meta_dict['labels'] =  labels[5:14]
        
    elif demography == 'median_income':
        demography_df = df.iloc[:,np.r_[1, 16:18]].T
        meta_dict['labels'] =  labels[14:16]

    elif demography == 'income':
        demography_df = df.iloc[:,np.r_[1, 18:23]].T
        meta_dict['labels'] =  labels[16:21]
        
    elif demography == 'occupation':
        demography_df = df.iloc[:,np.r_[1, 23:39]].T
        meta_dict['labels'] =  labels[21:]
    else:
        return jsonify('data not found')     
    

traces = {demography_df[i].to_list()[0] : demography_df[i].to_list()[1:] for i in demography_df}

myJSON = [meta_dict, traces]
myJSON

[{'demography': 'age',
  'labels': ['Age 0-5',
   'Age 6-10',
   'Age 11-15',
   'Age 16-20',
   'Age 20+',
   '9th Grade']},
 {'US': [0, 0, 0, 0, 0],
  'Mexico': [928523, 775875, 1485420, 2238925, 5753368],
  'Eastand Southeast Asia': [1500243, 946080, 926337, 941610, 4334258],
  'Central Asia': [38881, 21083, 20612, 17039, 34239],
  'South Asia': [1148791, 589390, 446589, 470990, 1013222],
  'Oceania': [61091, 28265, 32969, 34158, 89888],
  'Europe': [657376, 358714, 414540, 634284, 2783356],
  'Canada and Other North America': [151646, 67457, 62946, 87060, 457984],
  'Caribbean': [768542, 576927, 470081, 479039, 2169302],
  'Central America': [657604, 367185, 559321, 537020, 1469200],
  'South America': [706779, 292896, 366750, 626399, 1311556],
  'Middle East North Africa': [432957, 276164, 183893, 205151, 686733],
  'Sub Saharan Africa': [547733, 379302, 330441, 303174, 471820]}]

In [104]:
# @app.route("/api/v1.0/immigrants_by_county/<countries>/<years>/")

countries = 'Iran&Germany'
years = '2012&2014'

country_list = countries.split('&')

if ":" in years:
    
    year_range = years.split(':')
    year_range = [int(year) for year in year_range]
    year_list = list(range(year_range[0],year_range[1]))
    
elif years == 'all':
    
    a=1
    
else:
    
    year_list = years.split('&')
    year_list = [int(year) for year in year_list]


population_count = func.sum(details.admissions).label('Count')

if countries == 'all':
    
    country_filter = details.birth_country.isnot(None)
else:
    country_filter = details.birth_country.in_(country_list)

if years == 'all':
    
    year_filter = details.year.isnot(None)
    
else:
    
    year_filter = details.year.in_(year_list)


session = Session(engine)

Dataset = session.query(details.residence_county, counties.latitude, counties.longitude, population_count)\
.filter(country_filter)\
.filter(year_filter)\
.group_by(details.residence_county, counties.latitude, counties.longitude)\
.filter(details.residence_county == counties.county)\
.all()

# df = pd.read_sql_query(stmt, session.bind)


session.close

output_json = {
    
    'subject': ', '.join(country_list),
    'labels':['Count'],
    'locations': [[*row] for row in Dataset]
}

output_json

{'subject': 'Iran, Germany',
 'labels': ['Count'],
 'locations': [['Frederick', 39.470427, -77.397627, 10],
  ['Lee', 40.647588, -91.477157, 589],
  ['Jackson', 42.411782, -122.675797, 92],
  ['Washington', 35.844589, -76.572334, 739],
  ['Milwaukee', 43.017655, -87.481575, 202],
  ['Marion', 38.648396, -88.920221, 124],
  ['Somerset', 38.07445, -75.853323, 93],
  ['Kings', 36.072478, -119.81553, 1396],
  ['Clark', 34.053312, -93.176205, 1763],
  ['Johnson', 38.883907, -94.82233, 278],
  ['Clark', 37.970133, -84.144974, 1763],
  ['Montgomery', 37.189537, -95.742403, 3993],
  ['Nassau', 30.605926, -81.764929, 878],
  ['Lancaster', 40.041992, -76.250198, 130],
  ['Camden', 39.802352, -74.961251, 31],
  ['Clayton', 33.552242, -84.412977, 9],
  ['Bergen', 40.95909, -74.074522, 686],
  ['Summit', 40.87206, -110.968486, 74],
  ['Fairfield', 34.395669, -81.127001, 388],
  ['Allen', 36.75077, -86.192458, 13],
  ['Jefferson', 36.048479, -83.440966, 434],
  ['Johnson', 32.689831, -82.661354, 278

In [53]:
immigrants_by_country={
    'subject':'Iran'
    'labels':['count']
    'locations': [
        ['Orange', #name
        [25000],   #Values 
        ['long', 'lat'] #coordinates
        ]
    ]
}

SyntaxError: invalid syntax (<ipython-input-53-07038953b04c>, line 3)

In [73]:
years = "2012:2017"
year_range = years.split(':')
year_range = [int(year) for year in year_range]
year_list = list(range(year_range[0],year_range[1]))
year_list

[2012, 2013, 2014, 2015, 2016]

In [88]:
', '.join(country_list)


'Iran, Germany'

In [106]:
# @app.route("/api/v1.0/immigrants_by_state/<countries>/<years>/")

countries = 'Iran'
years = 'all'

country_list = countries.split('&')

if ":" in years:
    
    year_range = years.split(':')
    year_range = [int(year) for year in year_range]
    year_list = list(range(year_range[0],year_range[1]))
    
elif years == 'all':
    
    a=1
    
else:
    
    year_list = years.split('&')
    year_list = [int(year) for year in year_list]


population_count = func.sum(details.admissions).label('Count')

if countries == 'all':
    
    country_filter = details.birth_country.isnot(None)
else:
    country_filter = details.birth_country.in_(country_list)

if years == 'all':
    
    year_filter = details.year.isnot(None)
    
else:
    
    year_filter = details.year.in_(year_list)


session = Session(engine)

Dataset = session.query(details.residence, states.latitude, states.longitude, population_count)\
.filter(country_filter)\
.filter(year_filter)\
.group_by(details.residence, states.latitude, states.longitude)\
.filter(details.residence == states.name)\
.all()

session.close

output_json = {
    
    'subject': ', '.join(country_list),
    'labels':['Count'],
    'locations': [[*row] for row in Dataset]
}

output_json

{'subject': 'Iran',
 'labels': ['Count'],
 'locations': [['Oregon', 43.804133, -120.554201, 811],
  ['Idaho', 44.068202, -114.742041, 76],
  ['California', 36.778261, -119.417932, 78253],
  ['Arizona', 34.048928, -111.093731, 2357],
  ['Georgia', 32.157435, -82.907123, 3125],
  ['Washington', 47.751074, -120.740139, 3747],
  ['New York', 43.299428, -74.217933, 3484],
  ['Michigan', 44.314844, -85.602364, 963],
  ['District of Columbia', 38.905985, -77.033418, 315],
  ['Tennessee', 35.517491, -86.580447, 634],
  ['Alaska', 63.588753, -154.493062, 6],
  ['Massachusetts', 42.407211, -71.382437, 2248],
  ['Hawaii', 19.898682, -155.665857, 19],
  ['Nevada', 38.80261, -116.419389, 1408],
  ['Minnesota', 46.729553, -94.6859, 400],
  ['Missouri', 37.964253, -91.831833, 312],
  ['Virginia', 37.431573, -78.656894, 5316],
  ['Maryland', 39.045755, -76.641271, 4394],
  ['Indiana', 40.551217, -85.602364, 63],
  ['Pennsylvania', 41.203322, -77.194525, 762],
  ['Texas', 31.968599, -99.901813, 12719],

In [25]:
region = "Europe"

def ObtainCountries(region):

    session = Session(engine)

    Dataset = session.query(regions.country)\
    .filter(regions.region == region).all()
    
    session.close
    
    return [row.country for row in Dataset]

def Generate_lists(countries, years):

    if ":" in years:
    
        year_range = years.split(':')
        year_range = [int(year) for year in year_range]
        year_list = list(range(year_range[0],year_range[1]))
    
    elif years == 'all':
    
        a=1
    
    else:
    
        year_list = years.split('&')
        year_list = [int(year) for year in year_list]


    if countries == 'all':
    
        country_filter = details.birth_country.isnot(None)
        country_list = ['all']

    elif 'region' in countries:

        region = countries.split(":")[1]
        country_list = ObtainCountries(region)
        country_filter = details.birth_country.in_(country_list)



    else:
        country_list = countries.split('&')
        country_filter = details.birth_country.in_(country_list)

    if years == 'all':
    
        year_filter = details.year.isnot(None)
    
    else:
    
        year_filter = details.year.in_(year_list)

    return [country_filter, year_filter, country_list]

ObtainCountries("middle east")    

[]

In [57]:


countries = 'Iran'
years = 'all'

country_filter, year_filter, country_list =  Generate_lists(countries, years)

population_count = func.sum(details.admissions).label('Count')

session = Session(engine)

# , counties.latitude, counties.longitude

Dataset = session.query(details.residence, details.residence_county, counties.latitude, counties.longitude, population_count)\
.filter(country_filter)\
.filter(year_filter)\
.join(counties, and_(details.residence_county == counties.county, details.residence == counties.state))\
.group_by(details.residence, details.residence_county, counties.latitude, counties.longitude)\
.order_by(details.residence_county)\
.all()

# .join(counties)\
# .filter(details.residence_county == counties.county)\
# .filter(details.residence == counties.state)\

session.close

myJson =   {

    'subject': ', '.join(country_list),
    'labels':['Count'],
    'locations': [[*row] for row in Dataset]
}

myJson

{'subject': 'Iran',
 'labels': ['Count'],
 'locations': [['Idaho', 'Ada', 43.447861, -116.244456, 76],
  ['Colorado', 'Adams', 39.874325, -104.331872, 30],
  ['Florida', 'Alachua', 29.67574, -82.357221, 6],
  ['California', 'Alameda', 37.648081, -121.913304, 1624],
  ['New York', 'Albany', 42.588271, -73.974014, 61],
  ['Pennsylvania', 'Allegheny', 40.46892, -79.98092, 202],
  ['Alaska', 'Anchorage', 61.177549, -149.274354, 6],
  ['Maryland', 'Anne Arundel', 38.993374, -76.560511, 36],
  ['Minnesota', 'Anoka', 45.27411, -93.242723, 4],
  ['Colorado', 'Arapahoe', 39.644632, -104.331733, 234],
  ['Virginia', 'Arlington', 38.878337, -77.100703, 199],
  ['New Jersey', 'Atlantic', 39.469354, -74.633758, 4],
  ['Maryland', 'Baltimore', 39.443167, -76.616569, 502],
  ['New Jersey', 'Bergen', 40.95909, -74.074522, 454],
  ['Pennsylvania', 'Berks', 40.413957, -75.92686, 4],
  ['New Mexico', 'Bernalillo', 35.054002, -106.669065, 189],
  ['Texas', 'Bexar', 29.448671, -98.520147, 1227],
  ['Colora

In [51]:
abv_df = pd.read_csv("CSV/states_abv.csv")
counties_df = pd.read_csv("CSV/county coords.csv")

abv_df.head()

State_Name State
0     Alabama    AL
1      Alaska    AK
2     Arizona    AZ
3    Arkansas    AR
4  California    CA

In [50]:
counties_df.head()

State   County   Latitude  Longitude
0    AL  Autauga  32.536382 -86.644490
1    AL  Baldwin  30.659218 -87.746067
2    AL  Barbour  31.870670 -85.405456
3    AL     Bibb  33.015893 -87.127148
4    AL   Blount  33.977448 -86.567246

In [52]:
new_counties_df = pd.merge(counties_df,abv_df,on = "State")

In [53]:
new_counties_df.head()

State   County   Latitude  Longitude State_Name
0    AL  Autauga  32.536382 -86.644490    Alabama
1    AL  Baldwin  30.659218 -87.746067    Alabama
2    AL  Barbour  31.870670 -85.405456    Alabama
3    AL     Bibb  33.015893 -87.127148    Alabama
4    AL   Blount  33.977448 -86.567246    Alabama

In [54]:
new_counties_df.to_csv("CSV/counties.csv")

In [76]:
# @app.route("/api/v1.0/countries")

session = Session(engine)

country = distinct(details.birth_country)
Dataset = session.query(country).all()

session.close

myJson = [row[0] for row in Dataset]

myJson.sort()

myJson



['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 "China, People's Republic",
 'Colombia',
 'Congo, Democratic Republic',
 'Congo, Republic',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Curacao',
 'Cyprus',
 'Czechia',
 'Czechoslovakia (former)',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'French Polynesia',
 'Gabon',
 'Gambia',
 'Georgia

In [82]:
session = Session(engine)

country = distinct(regions.region)
Dataset = session.query(country).all()

session.close

myJson = [row[0] for row in Dataset]

myJson.sort()

myJson



['Canada and Other North America',
 'Caribbean',
 'Central America',
 'Central Asia',
 'East and Southeast Asia',
 'Europe',
 'Mexico',
 'Middle East-North Africa',
 'Oceania',
 'South America',
 'Sub-Saharan Africa',
 'Unknown']

In [86]:
# @app.route("/api/v1.0/diversity_by_state/<locations>/<years>/<top>")
# def immigrants_by_state(locations, years, top):
locations = "California"
years = "all"
top = "all"



country_filter, year_filter, country_list =  Generate_lists(locations, years)

population_count = func.sum(details.admissions).label('Count')

if top == 'all':
    top = 9000000

session = Session(engine)

Dataset = session.query(details.birth_country, population_count)\
.filter(country_filter)\
.filter(year_filter)\
.group_by(details.birth_country)\
.order_by(population_count.desc())\
.limit(int(top))\
.all()

session.close

json = {

'subject': ', '.join(country_list),
'labels':['Count'],
'locations': [[*row] for row in Dataset]
}

json

#return jsonify(json)

{'subject': 'California', 'labels': ['Count'], 'locations': []}